In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from seaborn import color_palette
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score, recall_score

%matplotlib inline

In [3]:
dados = pd.read_csv('dengue_oversample.csv',low_memory=False)
print(dados.shape)
dados.head()

(98459, 54)


,_id,tp_sexo,tp_gestante,tp_raca_cor,tp_escolaridade,co_uf_residencia,co_municipio_residencia,co_distrito_residencia,co_bairro_residencia,tp_classificacao_final,...,hematolog,hepatopat,renal,hipertensao,acido_pept,auto_imune,notificao_dias,idade,caso_internacao,train
0,"ObjectId(""5d10466a626562099323d893"")",M,6.0,9.0,10.0,26.0,261160.0,122.0,820.0,5,...,2.0,2.0,2.0,2.0,2.0,2.0,334.0,2.0,SIM,T
1,"ObjectId(""5d10466a626562099323d894"")",F,9.0,9.0,9.0,26.0,261160.0,119.0,841.0,5,...,2.0,2.0,2.0,2.0,2.0,2.0,365.0,12.0,SIM,T
2,"ObjectId(""5d10466a626562099323d895"")",M,6.0,4.0,NaN,26.0,261160.0,118.0,826.0,5,...,2.0,2.0,2.0,2.0,2.0,2.0,122.0,22.0,SIM,T
3,"ObjectId(""5d10466a626562099323d896"")",F,6.0,9.0,10.0,26.0,261160.0,122.0,820.0,1,...,2.0,2.0,2.0,2.0,2.0,2.0,61.0,4.0,SIM,T
4,"ObjectId(""5d10466a626562099323d897"")",F,9.0,9.0,NaN,26.0,261160.0,122.0,824.0,5,...,2.0,2.0,2.0,2.0,2.0,2.0,61.0,46.0,SIM,T


In [4]:
dados.columns

Index(['_id', 'tp_sexo', 'tp_gestante', 'tp_raca_cor', 'tp_escolaridade',
       'co_uf_residencia', 'co_municipio_residencia', 'co_distrito_residencia',
       'co_bairro_residencia', 'tp_classificacao_final',
       'tp_criterio_confirmacao', 'tp_autoctone_residencia', 'co_uf_infeccao',
       'co_pais_infeccao', 'co_municipio_infeccao', 'co_distrito_infeccao',
       'co_bairro_infeccao', 'no_bairro_infeccao', 'st_doenca_trabalho',
       'tp_evolucao_caso', 'dt_obito', 'dt_encerramento',
       'st_ocorreu_hospitalizacao', 'dt_internacao', 'co_uf_hospital',
       'co_municipio_hospital', 'co_unidade_hospital', 'nu_ddd_hospital',
       'nu_telefone_hospital', 'febre', 'mialgia', 'cefaleia', 'exantema',
       'vomito', 'nausea', 'dor_costas', 'conjutivite', 'artrite', 'artralgia',
       'petequia_n', 'leucopenia', 'laco', 'dor_retro', 'diabetes',
       'hematolog', 'hepatopat', 'renal', 'hipertensao', 'acido_pept',
       'auto_imune', 'notificao_dias', 'idade', 'caso_internacao

In [5]:
treino = dados[dados.train=='T'].drop('train',axis=1)
print(treino.shape)
print(treino.caso_internacao.value_counts())
teste = dados[dados.train=='F'].drop('train',axis=1)
print(teste.shape)
print(teste.caso_internacao.value_counts())

(68920, 53)
SIM    34460
NAO    34460
Name: caso_internacao, dtype: int64
(29539, 53)
NAO    14879
SIM    14618
29         3
40         3
0          2
47         2
2          2
82         2
95         1
15         1
31         1
88         1
14         1
51         1
60         1
68         1
70         1
63         1
55         1
79         1
59         1
67         1
36         1
22         1
43         1
62         1
61         1
48         1
33         1
49         1
30         1
9          1
4          1
Name: caso_internacao, dtype: int64


In [6]:
treinoCP = sm.add_constant(treino.copy())
treinoCP['Ip'] = treinoCP.caso_internacao.map(lambda x: x=='SIM' and 1 or 0)
model = smf.logit("Ip ~ febre + mialgia + cefaleia + exantema + vomito + nausea + dor_costas + conjutivite + artrite + artralgia + petequia_n + leucopenia + laco + dor_retro + diabetes + hematolog + hepatopat + renal + hipertensao + acido_pept + idade",data=treinoCP).fit()

model.summary()

Optimization terminated successfully.
         Current function value: 0.573278
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     Ip   No. Observations:                68920
Model:                          Logit   Df Residuals:                    68898
Method:                           MLE   Df Model:                           21
Date:                Mon, 24 Jun 2019   Pseudo R-squ.:                  0.1729
Time:                        15:09:53   Log-Likelihood:                -39510.
converged:                       True   LL-Null:                       -47772.
                                        LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      24.0596      3.101      7.758      0.000      17.981      30.138
febre[T.2]     -1.9424      0.050    -39.052      0.000      -2.040      -1.845
mialgia        -0.8939      0.050    -17.956      0.000      -0.991      -0.796
cefaleia       -0.4321      0.051     -8.516      0.000      -0.532      -0.333
exantema        0.2671      0.056      4.800      0.000       0.158       0.376
vomito         -1.1813      0.073    -16.242      0.000      -1.324      -1.039
nausea          0.1706      0.069      2.460      0.014       0.035       0.307
dor_costas     -1.2166      0.153     -7.951      0.000      -1.517      -0.917
conjutivite    -0.8109      0.292     -2.778      0.005      -1.383      -0.239
artrite         0.2820      0.128      2.200      0.028       0.031       0.533
artralgia       0.6513      0.053     12.269      0.000       0.547       0.755
petequia_n     -1.5961      0.218     -7.311      0.000      -2.024      -1.168
leucopenia     -1.8087      0.245     -7.391      0.000      -2.288      -1.329
laco           -0.2779      0.461     -0.602      0.547      -1.182       0.627
dor_retro       0.5776      0.073      7.947      0.000       0.435       0.720
diabetes       -0.8459      0.412     -2.051      0.040      -1.654      -0.038
hematolog       0.6588      0.600      1.099      0.272      -0.516       1.834
hepatopat      -0.1045      0.452     -0.231      0.817      -0.991       0.782
renal          -1.3283      1.096     -1.212      0.225      -3.476       0.819
hipertensao    -2.5772      0.398     -6.471      0.000      -3.358      -1.797
acido_pept     -0.9465      0.539     -1.756      0.079      -2.003       0.110
idade           0.0064      0.000     15.740      0.000       0.006       0.007
===============================================================================
"""

In [7]:
model.predict(teste)

PatsyError: Error converting data to categorical: observation with value '29-06-2016' does not match any of the expected levels (expected: ['1', '2'])
    Ip ~ febre + mialgia + cefaleia + exantema + vomito + nausea + dor_costas + conjutivite + artrite + artralgia + petequia_n + leucopenia + laco + dor_retro + diabetes + hematolog + hepatopat + renal + hipertensao + acido_pept + idade
         ^^^^^